# Implementing UNet Architecture From Scratch Using PyTorch

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import torchvision as TV
import matplotlib.pyplot as plt

# UNet Model Architecture:

In [ ]:
# Define custom UNet model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Encoder Block
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.pool = self.MaxPool2d(kernel_size=2, stride=2)

        # Transposed Convolutions Block
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        # Decoder Block
        self.dec1 = self.conv_block(512, 256)
        self.dec2 = self.conv_block(256, 128)
        self.dec3 = self.conv_block(128, 64)

        self.out = nn.Conv2d(64, out_channels, kernel_size=1)


    # Define Convolutions block for reusability
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels),
            nn.ReLU(inplace=True)
        )

    # Forward Method
    def forward(self, x):

        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(1))
        x3 = self.enc3(self.pool(x2))
        x4 = self.enc4(self.pool(x3))

        x = self.upconv3(x4)
        x = T.cat([x, x3], dim=1)
        x = self.dec1(x)

        x = self.upconv2(x)
        x = T.cat([x, x2], dim=1)
        x = self.dec2(x)

        x = self.upconv1(x)
        x = T.cat([x, x1], dim=1)
        x = self.dec3(x)
        
        return self.out(x)
